<h4>1. Which linear regression training algorithm can you use if you have a training set with millions of features?</h4>

Stochastic Gradient Descent or Mini-batch Gradient Descent, and maybe Batch Gradient Descent if the training set fits to memory. You cannot use the Normal Equation or SVD because the computational complexity grows quickly (more than quadratically) with the number of features.

<h4>2. Suppose the features in your training set have diferent scales. Which algorithms might suffer from this, and how? What can you do about it?</h4>

The cost function will have the shape of an elongated bowl, so the Gradient Descent algorithms will take a long time to converge. To solve this you should scale the data tbefore training the model. 

<h4>3. Can gradient descent get stuck in a local minimum when training a logistic regression model?</h4>

Gradient Descent cannot get stuck in a local minimum when training a Logistic Regresssion model because the cost function is convex.

<h4>4. Do all gradient descent algorithms lead to the same model, provided you let them run long enough?</h4>

If the optimisation problem is convex (such as Linear Regression or Logistic Regression), and assuming the learning rate is not too high, then all Gradient Descent algorithms will approach the global optimum and end up producing fairly similar models. 

However, unless you gradually reduce the learning rate, Stochastic Gradient Descent and Mini-batch Gradient Descent will never truly converge; instead, they will keep jumping back and forth around the global optimum. This means that even if you let them run for a very long time, these Gradient Descent algorithms will produce slightly different models.

<h4>5. Suppose you use batch gradient descent and you plot the validation error at every epoch. If you notice that the validation error consistently goes up, what is likely goiong on? How can you fix this?</h4>

If the validation error goes up at every epoch, then one possibility is that the learning rate is too high and the algorithm is diverging. If the training error also goes up, then this is clearly the problem and you should reduce the learning rate. However, if the training error is not going up, then your model is overfitting the trainging data and you should stop training.

<h4>6. Is it a good idea to stop mini-batch gradient descent immediately when the validation error goes up?</h4>

Due to their random nature, neither Stochastic Gradient Descent nor Mini-batch Gradient Descent is guaranteed to make progress at every single training iteration. So if you immediately stop training when the validation error goes up, you may stop much too early; before the optimum is reached.

A better option is to save the model at regular intervals; then, when it has not improved for a long time (meaning it will probably never beat the record), you can revert to the best model you have saved.

<h4>7. Which gradient descent algorithm (among those discussed) will reach the vicinity of the optimal solution the fastest? Which will actually converge? How can you make the others converge as well? </h4>

Stochastic Gradient Descent has the fastest training iteration since it considers only one training instance at a time, so it is generally the first to reach the vicinity of the global optimum (or Mini-batch GD with a very small mini-batch size). 

However, only Batch Gradient Descent will actually converge, given enough training time. As mentioned, Stochastic Gradient Descent and Mini-batch Gradient Descent will bounce around the optimum, unless you gradually reduce the learning rate.

<h4>8. Suppose you are usuing polynomial regression. You plot the learning curves and you notice that there is a large gap between the training error and the validatoin error. What is happening? What are three ways to solve this?</h4>

If the validation error is much higher than the training error, this is likely because your model is overfitting the training data. One way to try to fix this is to reduce the polynomial degree: a model with fewer degrees of freedom is less likely to overfit.

Another thing you can try is to regularise the model - for example, by adding an $\ell_{2}$ penalty (Ridge) or an $\ell_{1}$ penalty (Lasso) to the cost function. This will also reduce the degrees of freedom of the model.

Lastly, you can icnrease the size of the training set.

<h4>9. Suppose you are using ridge regression and you notice that the training error and the validation error are almost equal and fairly high. Would you say that the model suffers from high bias or high variance? Should you increase the regularisation hyperparameter $\alpha$ or reduce it?</h4>

The model is most likely underfitting the training set, which means it has a high bias. You should try reducing the regularisation hyperparameter.

<h4>10. Why would you want to use:
a. Ridge regression instead of plain linear regression?
b. Lasso instead of ridge regression?
c. Elastic net instead of lasso regression?
</h4>

<ul>
    <li>A model with some regularisation typically performs better than a medoel without any regularisation, so you should generally prefer Ridge Regression over plain Linear Regression.</li>
    <li>Lasso Regression uses an l_1 penalty, which tends to push the weights down to exactly zero. This leads to sparse models, where all weights are zero except for the most important weights. This is a way to perform feature selection automatically, which is good if you suspec that only a few features actually matter. When you aren't sure, you should prefer Ridge Regression.</li>
    <li>Elastic Net is generally preferred over Lasso sicne Lasso may behave erratically in some cases (when several features are strongly correlated or when there are more features than training instances). However, it does add an extra hyperparameter to tune. If you want Lasso without the erratic behaviour, you can just Elastic Net with an l1_ratio close to 1.</li>
</ul>

<h4>11. Suppose you want to classify pictures as outdoor/indoor and daytime/nighttime. Should you implement two logistic regression classifiers or one softmax regression classifier?</h4>

Since these are not exclusive features (i.e. all four combinations are possible) you should train two Logistic Regression classifiers.

<h4>12. Implement batch gradient descent with early stopping for softmax regression without using Scikit-Learn, only NumPy. Use it on a classification task such as the iris dataset.</h4>

In [1]:
from sklearn.datasets import load_iris

iris = load_iris(as_frame = True)

In [2]:
X = iris.data[["petal length (cm)", "petal width (cm)"]].values
y = iris["target"].values

We need to add the bias term for every instance ($x_{0}=1$).

In [3]:
import numpy as np

X_with_bias = np.c_[np.ones(len(X)), X]

Next, we need to split the dataset into a training set, a validation set, and a test set.

In [4]:
test_ratio = 0.2
validation_ratio = 0.2
total_size = len(X_with_bias)

test_size = int(total_size * test_ratio)
validation_size = int(total_size * validation_ratio)
train_size = total_size - test_size - validation_size

np.random.seed(42)
rnd_indices = np.random.permutation(total_size)

X_train = X_with_bias[rnd_indices[:train_size]]
y_train = y[rnd_indices[:train_size]]
X_valid = X_with_bias[rnd_indices[train_size:-test_size]]
y_valid = y[rnd_indices[train_size:-test_size]]
X_test = X_with_bias[rnd_indices[-test_size:]]
y_test = y[rnd_indices[-test_size:]]

The targets are currently class indicies (0, 1, or 2), but we need target class probabilities to train the Softmax Regression model.

Each instance will have target class probabilities equal to 0.0 for all classes except the target class which will have a probability of 1.0.

Next, we write a small function to conver the vector of class indices into a matrix containing a one-hot vector for each instance. 

In [5]:
def to_one_hot(y):
    return np.diag(np.ones(y.max() + 1))[y]

In [6]:
to_one_hot(y_train[:10])

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.]])

Next, we create the target class probabilities matrix for the training set and the test set:

In [7]:
Y_train_one_hot = to_one_hot(y_train)
Y_valid_one_hot = to_one_hot(y_valid)
Y_test_one_hot = to_one_hot(y_test)

Now, we scale the inputs. We compute the mean and the standard deviation of each feature on the trainign set (except for the bias feature), then we centre and scale each feature in the training set, and the test set:

In [8]:
mean = X_train[:, 1:].mean(axis=0)
std = X_train[:, 1:].std(axis=0)
X_train[:, 1:] = (X_train[:, 1:] - mean) / std
X_valid[:, 1:] = (X_valid[:, 1:] - mean) / std
X_test[:, 1:] = (X_test[:, 1:] - mean) / std

Next, we implement the softmax function.

In [9]:
def softmax(logits):
    exps = np.exp(logits)
    exp_sums = exps.sum(axis=1, keepdims=True)
    return exps / exp_sums

In [10]:
n_inputs = X_train.shape[1]  # == 3 (2 features plus the bias term)
n_outputs = len(np.unique(y_train))  # == 3 (there are 3 iris classes)

Now, we train the model.

In [11]:
eta = 0.5
n_epochs = 5001
m = len(X_train)
epsilon = 1e-5

np.random.seed(42)
Theta = np.random.randn(n_inputs, n_outputs)

for epoch in range(n_epochs):
    logits = X_train @ Theta
    Y_proba = softmax(logits)
    if epoch % 1000 == 0:
        Y_proba_valid = softmax(X_valid @ Theta)
        xentropy_losses = -(Y_valid_one_hot * np.log(Y_proba_valid + epsilon))
        print(epoch, xentropy_losses.sum(axis=1).mean())
    error = Y_proba - Y_train_one_hot
    gradients = 1 / m * X_train.T @ error
    Theta = Theta - eta * gradients

0 3.7085808486476917
1000 0.14519367480830644
2000 0.1301309575504088
3000 0.12009639326384539
4000 0.11372961364786884
5000 0.11002459532472425


Now we can look at the model parameters.

In [12]:
Theta

array([[ 0.41931626,  6.11112089, -5.52429876],
       [-6.53054533, -0.74608616,  8.33137102],
       [-5.28115784,  0.25152675,  6.90680425]])

We can predictions for the validation set and check the accuracy of the score:

In [13]:
logits = X_valid @ Theta
Y_proba = softmax(logits)
y_predict = Y_proba.argmax(axis=1)

accuracy_score = (y_predict == y_valid).mean()
accuracy_score

np.float64(0.9333333333333333)